In [ ]:
def install_earth_packages():
    """
    Install essential packages for accessing NASA Earthdata and handling NetCDF data.
    """
    import subprocess
    import sys

    packages = ["earthaccess", "netCDF4", "requests"]
    for pkg in packages:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])


In [ ]:
#User Coordinates

#JSON
{
  "latitude": 7.8731,
  "longitude": 80.7718
}


In [ ]:
#---Global Variables---

#---Openweather API and URL----
OPENWEATHER_API_KEY = "your key"
openweather_url = "https://api.openweathermap.org/data/2.5/weather"

#---NASA Earthdata username and password---
nasaearthdata_username = "username"
nasaearthdata_password = "password"

#---Longitude and Latitude---
lat, lon = 1.11111, 10.11111 #Change as per your coordinates

#---Openmeteo url---
openmeteo_url = "https://api.open-meteo.com/v1/forecast"

In [ ]:
#---Openweather realtime weather feedback
def openweather(OPENWEATHER_API_KEY, openweather_url):

    latitude = 6.823096
    longitude = 79.943463


    params = {
        "lat": latitude,
        "lon": longitude,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric"
    }

    response = requests.get(openweather_url, params=params)
    data = response.json()

    if "main" in data and "wind" in data:
        temp = data["main"]["temp"]
        humidity = data["main"]["humidity"]
        description = data["weather"][0]["description"]
        wind_speed = data["wind"]["speed"]

        print(f" Temperature: {temp}°C")
        print(f" Humidity: {humidity}%")
        print(f" Wind Speed: {wind_speed} m/s")
        print(f" Weather: {description}")
    else:
        print("API Error or unexpected response:")
        print(data)

    return temp, humidity, description, wind_speed

In [ ]:
#netrc generation for NASA EARTHDATA LOGIN
def NASA_netrc(nasaearthdata_username, nasaearthdata_password):
    netrc_content = f"""machine urs.earthdata.nasa.gov
    login {nasaearthdata_username}
    password {nasaearthdata_password}
    """

    # Write .netrc file to home directory
    with open("/root/.netrc", "w") as f:
        f.write(netrc_content)

    # Set permissions so it’s only readable by you (required)
    !chmod 600 /root/.netrc

    print("✅ .netrc file created and permissions set")


In [ ]:
def NASA_EARTHDATA_AUTHENTICATION():

    auth = Auth()

    # This should now work silently using .netrc without prompt
    auth.login()
    print("✅ Authenticated with NASA Earthdata!")


In [ ]:
def NASA_EARTHDATA_SOIL_MOISTURE(lat=6.823096, lon=79.943463):
    """
    Fetches soil moisture from NASA SMAP SPL3SMP product for given lat/lon.
    """

    collection_shortname = "SPL3SMP"  # SMAP L3 Soil Moisture product

    # Define time range (last 5 days)
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=5)

    # Get the data collection metadata
    collections = DataCollections().short_name(collection_shortname).get()

    # Define bounding box BEFORE using lat/lon
    bbox = [lon - 0.1, lat - 0.1, lon + 0.1, lat + 0.1]

    # Search for granules
    granules = (
        DataGranules()
        .short_name(collection_shortname)
        .bounding_box(*bbox)
        .temporal(start_date.isoformat() + "Z", end_date.isoformat() + "Z")
        .get()
    )

    if not granules:
        print("No granules found for this area and time")
        return

    print(f"Found {len(granules)} granules")
    file_url = granules[0].data_links()[0]
    print("First granule URL:", file_url)

    # Download the HDF5 file
    response = requests.get(file_url)
    response.raise_for_status()

    with h5py.File(BytesIO(response.content), "r") as f:
        # Use AM retrieval data
        group = f["Soil_Moisture_Retrieval_Data_AM"]

        latitudes = group["latitude"][:]
        longitudes = group["longitude"][:]
        soil_moisture = group["soil_moisture"][:]

        # Mask invalid values
        valid_mask = soil_moisture != -9999

        # Compute distances only for valid points
        dist = np.full(soil_moisture.shape, np.inf)
        dist[valid_mask] = np.sqrt((latitudes[valid_mask] - lat)**2 + (longitudes[valid_mask] - lon)**2)

        # Find closest valid point
        i, j = np.unravel_index(np.argmin(dist), dist.shape)

        value = soil_moisture[i, j]
        print(f"🌱 Soil moisture (AM) at {lat},{lon}: {value:.3f} m³/m³")

    return value

In [ ]:
#---Google Earth Engine---
def earth_engine_authentication():
    ee.Authenticate()  # Follow the link, authenticate with your Google account
    ee.Initialize(project='your ID')


In [ ]:
'''def MODIS():

    # Define your location point
    point = ee.Geometry.Point([lon, lat])

    # MODIS Vegetation Indices (MOD13Q1) - 16-day, 250m resolution
    modis_veg = ee.ImageCollection('MODIS/006/MOD13Q1') \
                  .filterBounds(point) \
                  .filterDate('2025-01-01', '2025-06-21') \
                  .select(['NDVI', 'EVI'])

    # MODIS Land Surface Temperature (MOD11A2) - 8-day, 1km resolution
    modis_lst = ee.ImageCollection('MODIS/006/MOD11A2') \
                  .filterBounds(point) \
                  .filterDate('2025-01-01', '2025-06-21') \
                  .select(['LST_Day_1km'])

    # Get median composite for NDVI and EVI
    veg_img = modis_veg.median()

    # Get median composite for LST
    lst_img = modis_lst.median()

    # Sample the NDVI, EVI, LST values at your point
    veg_values = veg_img.sample(point, scale=250).first().getInfo()
    lst_values = lst_img.sample(point, scale=1000).first().getInfo()

    print("MODIS Vegetation Indices at your location:")
    print(f"NDVI: {veg_values['properties']['NDVI']} (scale factor: 0.0001, multiply to get real value)")
    print(f"EVI: {veg_values['properties']['EVI']} (scale factor: 0.0001)")

    print("\nMODIS Land Surface Temperature at your location:")
    print(f"LST_Day_1km: {lst_values['properties']['LST_Day_1km']} (scale factor: 0.02 Kelvin)")'''


In [ ]:

def openmeteo(openmeteo_url):
    # Coordinates and parameters

    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": "shortwave_radiation",
        "timezone": "Asia/Colombo"  # Make sure it's local time
    }

    # Fetch data
    resp = requests.get(openmeteo_url, params=params).json()

    # Get today's date
    today_str = datetime.now().strftime("%Y-%m-%d")

    # Filter and print today's data
    times = resp["hourly"]["time"]
    radiation = resp["hourly"]["shortwave_radiation"]

    print(f"Shortwave Radiation for Today ({today_str}):")
    for t, rad in zip(times, radiation):
        if t.startswith(today_str):
            print(f"{t[-5:]} → {rad:.1f} W/m²")


In [ ]:
#import packages
from earthaccess import DataCollections, DataGranules
import ee
from datetime import datetime, timedelta
import h5py
import numpy as np
import requests
from io import BytesIO
import ee
import requests


In [ ]:
#---Pipeline---
install_earth_packages()
temp, humidity, description, wind_speed = openweather(OPENWEATHER_API_KEY, openweather_url)
NASA_EARTHDATA_AUTHENTICATION()
NASA_EARTHDATA_SOIL_MOISTURE(lat=1.1111, lon=10.11111) #Change as per your coordinates
#earth_engine_authentication()
#MODIS()
openmeteo(openmeteo_url)
